This jupyter notebook is used to correctly write out the PSF_MASK fits files - this jupyter nb is needed because Mathematica cannot properly write FITS files.

In [ ]:
from astropy.io import fits
import os
import glob
import numpy as np
import re
from shutil import copyfile
import matplotlib.pyplot as plt

In [ ]:
# Relevant directories
working_dir = os.getcwd()+'/'
old_file_path = working_dir + 'psf_masks_old/' # old psfmasks
new_file_path_mathematica = working_dir + 'psf_masks_new/mathematica_out/' # new psfmasks produced by Mathematica, which are missing the correct headers
new_file_path = working_dir + 'psf_masks_new/' # finalized psfmask files with corrected headers

In [ ]:
# Relevant filepaths
old_psf_mask_paths = np.sort(glob.glob(old_file_path+'*.fits')) 
new_psf_mask_paths_mathematica = np.sort(glob.glob(new_file_path_mathematica+'*_pre.fits')) 

In [ ]:
print(old_psf_mask_paths)
print("=======================================")
print(new_psf_mask_paths_mathematica)

In [ ]:
hdus_old = [fits.open(path) for path in old_psf_mask_paths]
hdus_mathematica = [fits.open(path) for path in new_psf_mask_paths_mathematica]

In [ ]:
kk=3
hdus_old[kk].info()
print("=======================================")
hdus_mathematica[kk].info()

In [ ]:
# Assign the Mathematica-generated files with the correct header formatting using the old ref files
for i in range(0, len(hdus_old)):
    hdus_old[i][1] = fits.ImageHDU(data = hdus_mathematica[i][0].data, header = hdus_old[i][1].header)
    new_fnames = [re.sub('_pre', '_new', file) for file in new_psf_mask_paths_mathematica]
    hdus_old[i].writeto(new_fnames[i], overwrite=True)

# Copy the new psfmask ref files to the 'psf_mask_new' directory   
for file_path in glob.glob(os.path.join(new_file_path_mathematica, '*new.fits')):
    path, filename = os.path.split(file_path)
    new_filename = filename.replace('_new.fits', '.fits')
    new_psf_mask_path = new_file_path + new_filename
    copyfile(file_path, new_psf_mask_path)

In [ ]:
# Check the formatting of the new fits files
new_psf_mask_paths = np.sort(glob.glob(new_file_path + '*.fits'))
hdus_new = [fits.open(path) for path in new_psf_mask_paths]

kk = 3
hdus_old[kk].info()
print("=======================================")
hdus_new[kk].info()

In [ ]:
# Open each ref file and update the primary header
for i, file in enumerate(new_psf_mask_paths):
    f = fits.open(new_psf_mask_paths[i], mode='update')
    hdr = f[0].header
    hdr['HISTORY'][0] = 'Correction of previous PSFMASK ref files, by B. Hagan & B. Nickson'
    hdr['HISTORY'][1] = 'DOCUMENT: N/A'
    hdr['HISTORY'][2] = 'SOFTWARE: Mathematica 12'
    hdr['HISTORY'][5] = "DIFFERENCES: Previous PSFMASK was the wrong shape and incorrectly centered around the coronagraphic obstructions. New mask is a simple binary mask with a central hole equal to the size of the coronagraph's IWA."
    hdr['AUTHOR'] = 'Coronagraphs Working Group (B. Nickson)'
    hdr['EXP_TYPE'] = 'N/A'
    f.close()